In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import time

In [23]:
# PATH = "C:\\Users\\Skelet0N_0FF\\Desktop\\parsing\\driver\\geckodriver.exe"
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

try:
    driver.get("https://mandarinbean.com/new-hsk-1-word-list/")
    headers = [header.get_attribute("innerHTML") for header in driver.find_elements(By.TAG_NAME, "strong")]
    final_data = pd.DataFrame(columns=headers)
    def get_table(webpage):     # Берёт страницу и собирает с неё все элементы таблицы (в тегах <td>) в датафрейм
        driver.get(webpage)
        element = WebDriverWait(driver, 10).until(          # Ожидание прогрузки элементов в тегах <td>
            EC.presence_of_element_located((By.TAG_NAME, "td"))
        )
        temp_data = [value.get_attribute("innerHTML") for value in driver.find_elements(By.TAG_NAME, "td")]
        data = []
        for i in range(4, len(temp_data), 4):         # Не уверен, что написал эту штуку правильно ПРОВЕРЬ
            data.append([td.replace("&nbsp;", " ") for td in temp_data[i-4:i]])
        return pd.DataFrame(data=data, columns=headers)

    for i in range(1, 4):
        final_data = pd.concat(objs=[final_data, get_table(f"https://mandarinbean.com/new-hsk-{i}-word-list/")]) 
        
    final_data.drop(labels="No", axis=1, inplace=True)
    final_data.to_csv(path_or_buf=".\\pinyin.csv", index=False)
except Exception as e:
    print(f"{e}\n\nThe process will terminate in 10 seconds...")
    time.sleep(10)
finally:
    driver.close()
    driver.quit()

In [24]:
data = pd.read_csv("./pinyin.csv")
data.head()

,Chinese,Pinyin,English
0,爱,ài,love
1,爱好,ài hào,hobby
2,八,bā,eight
3,爸爸｜爸,bàba ｜ bà,dad
4,吧,ba,(interjection particle)
